In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime

In [ ]:
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
    "Referer": "https://www.google.com/"
}

In [ ]:
food_commonhealth_url = "https://www.commonhealth.com.tw/channel/48?page={}&tab=all"

df = pd.DataFrame(columns=["date","title","url","watch","content"])

for page in range(1,34):
    print(page)
    fc_res = requests.get(food_commonhealth_url.format(page), headers=headers)
    fc_soup = BeautifulSoup(fc_res.text, "html.parser")
    fc_new_soup = fc_soup.select("div[class='tab__target']")[0]
    fc_tmp_titles = fc_new_soup.select("div[class='title']")
    fc_urls = fc_soup.select("a[class='recommend recommend--channels']")

    for i,j in zip(range(1,10),range(0,9)):   #每頁扣除第一篇還有九篇

        fc_title = fc_tmp_titles[i].text
        fc_article_url = fc_urls[j]["href"]
        print(fc_title)
        print(fc_article_url)
        fc_article_res = requests.get(fc_article_url,headers=headers)
        fc_article_soup = BeautifulSoup(fc_article_res.text, "html.parser")        
        fc_article = fc_article_soup.select("div[class='essay']")
        if len(fc_article_soup.select("div[class='info__line info__line--view']")) == 0:
            pass
        else:
            tmp_watch = fc_article_soup.select("div[class='info__line info__line--view']")[0].text
            fc_watch = re.sub(u"([^\u0030-\u0039])","",tmp_watch)
            print(fc_watch)
            tmp_date = fc_article_soup.select("span[id='publish_time']")[0].text
            date = datetime.strptime(tmp_date, "%Y/%m/%d")
            print(date)
            if len(fc_article) == 0:
                pass
            else:
                print(fc_article[0].text)

                row = pd.DataFrame([[date,fc_title,fc_article_url,fc_watch,fc_article[0].text]],columns=["date","title","url","watch","content"])
                df = df.append(row, ignore_index = True)
    if page % 3 == 0:
        time.sleep(12)
    



In [ ]:
edh_url = "https://www.edh.tw/category/218/index/{}"

for page in range(1,20): 
    edh_res = requests.get(edh_url.format(page),headers)
    edh_soup = BeautifulSoup(edh_res.text, "html.parser")
    edh_tmp_soup = edh_soup.select("div[class='article-grid clear']")
    for i in range(2):     # 文章分為兩部分，分別爬取
        
        edh_new_soup = edh_tmp_soup[i]
        if len(edh_new_soup) == 0:    #如無內容則 PASS
            pass
        for j in edh_new_soup.select("a[class='detail']"):
            edh_title = j.select("h3[class='title']")[0].text
            edh_article_url = "https://www.edh.tw/" + j["href"] 
            print(edh_title)
            print(edh_article_url)
            watch_res = requests.get(edh_article_url, headers)
            watch_soup = BeautifulSoup(watch_res.text, "html.parser")
            tmp_watch = watch_soup.select("span[class='number']")[0].text
            edh_watch = re.sub(u"([^\u0030-\u0039])","",tmp_watch)
            print(edh_watch)
            
            tmp_date = watch_soup.select("span[class='date']")[0].text
            date = datetime.strptime(tmp_date, "%Y-%m-%d")

            edh_article = " "
            for k in range(1,3):
                edh_tmp_article_url = edh_article_url + "/{}"
                edh_article_res = requests.get(edh_tmp_article_url.format(k),headers)
                edh_article_soup = BeautifulSoup(edh_article_res.text, "html.parser")
                edh_tmp_article = edh_article_soup.select("div[id='article_page']")

                if len(edh_tmp_article) == 0:
                    pass
                else:
                    edh_article += edh_tmp_article[0].text
            print(edh_article)
            
            row = pd.DataFrame([[date,edh_title,edh_article_url,edh_watch,edh_article]],columns=["date","title","url","watch","content"])
            df = df.append(row, ignore_index = True)
            
    if page % 3 == 0:
        time.sleep(12)
            

In [ ]:
df = df.sort_values(by = ["date"], ignore_index = True)

In [ ]:
df.to_csv("./article_by_date.csv", encoding = "utf-8-sig", index=False)

In [ ]:
df_no_content = df.drop(columns=["content"])

In [ ]:
df_no_content.to_csv("./article_no_content.csv", index=False)

In [ ]:
df = pd.read_csv("./article_no_content.csv")